In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [9]:
from collections import Counter
import pandas as pd
import numpy as np
import nltk
from ipywidgets import Output
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import LdaModel
from gensim.models import Phrases
from itertools import chain
from gensim.corpora import Dictionary
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from itertools import chain
import networkx as nx

In [3]:
%%time
train = pd.read_csv('../data/external/kaggle/train.csv')
test = pd.read_csv('../data/external/kaggle/test.csv')
y = pd.read_csv('../data/external/kaggle/train.csv')['target'].values
tokenizer = RegexpTokenizer(r'\w+')


from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as SKLEARN_STOPWORDS
from spacy.lang.en.stop_words import STOP_WORDS as SPACY_STOPWORDS
from nltk.corpus import stopwords

SKLEARN_STOPWORDS = set(SKLEARN_STOPWORDS)
out = Output()
with out:
    nltk.download('stopwords')
NLTK_STOPWORDS = set(stopwords.words('english'))
print(f'Loaded {len(NLTK_STOPWORDS)} stopwords from NLTK')
print(f'Loaded {len(SPACY_STOPWORDS)} stopwords from SPACY')
print(f'Loaded {len(SKLEARN_STOPWORDS)} stopwords from SKLEARN')
stop_words = list(set.union(*[SKLEARN_STOPWORDS, SPACY_STOPWORDS, NLTK_STOPWORDS]))
print('----------------------------------')
print(f'{len(stop_words)} combined stopwords')

Loaded 179 stopwords from NLTK
Loaded 326 stopwords from SPACY
Loaded 318 stopwords from SKLEARN
----------------------------------
409 combined stopwords
CPU times: user 204 ms, sys: 20.8 ms, total: 224 ms
Wall time: 601 ms


In [4]:
%%time
corpus = train['text'].values
corpus = [doc.lower() for doc in corpus]
corpus = [tokenizer.tokenize(doc) for doc in corpus]
corpus = [[token for token in doc if (not token.isnumeric() and len(token) > 1)] for doc in corpus]
corpus= [[token for token in doc if (not token in stop_words)] for doc in corpus]
bigram = Phrases(corpus, min_count=1)
trigram = Phrases(bigram[corpus], min_count=1)
fourgram = Phrases(trigram[corpus], min_count=1)
bigrams = []
trigrams = []
fourgrams = []
for doc in corpus:
    b = [b for b in bigram[doc] if b.count('_') == 1]
    t = [t for t in trigram[bigram[doc]] if t.count('_') == 2]
    f = [f for f in fourgram[trigram[bigram[doc]]] if f.count('_') == 3]
    bigrams.extend(b)
    trigrams.extend(t)
    fourgrams.extend(f)

CPU times: user 3.06 s, sys: 2.13 ms, total: 3.06 s
Wall time: 3.06 s


In [5]:
Counter(bigrams).most_common(10)

[('û_', 77),
 ('suicide_bomber', 49),
 ('burning_buildings', 48),
 ('liked_youtube', 42),
 ('heat_wave', 31),
 ('california_wildfire', 31),
 ('natural_disaster', 30),
 ('prebreak_best', 30),
 ('mass_murder', 30),
 ('looks_like', 28)]

In [6]:
Counter(trigrams).most_common(10)

[('û__http', 150),
 ('liked_youtube_video', 35),
 ('û__https', 33),
 ('black_hat_http', 17),
 ('pamela_geller_http', 15),
 ('ambulance_helicopter_crash', 14),
 ('bigger_projected_http', 14),
 ('read_ebay_http', 12),
 ('subreddits_banned_quarantined', 11),
 ('hiroshima_atomic_bombing', 10)]

In [7]:
Counter(fourgrams).most_common(10)

[('latest_homes_razed_northern', 27),
 ('families_sue_legionnaires_families', 25),
 ('watch_airport_swallowed_sandstorm', 24),
 ('wreckage_conclusively_confirmed_mh370', 24),
 ('affected_fatal_outbreak_legionnaires', 22),
 ('suicide_bomber_detonated_bomb', 22),
 ('ignition_knock_detonation_sensor', 20),
 ('obama_declares_disaster_typhoon', 20),
 ('malaysia_pm_investigators_families', 20),
 ('video_picking_bodies_water', 18)]

In [8]:
df = pd.DataFrame(Counter(bigrams).most_common()).rename(columns={0:'bigrams', 1:'weight'})
df[['a', 'b']] = df['bigrams'].str.split(pat='_', expand=True)
df = df[['a','b','weight']]
df = df.replace('', np.nan).dropna().reset_index(drop=True)
df

,a,b,weight
0,suicide,bomber,49
1,burning,buildings,48
2,liked,youtube,42
3,heat,wave,31
4,california,wildfire,31
...,...,...,...
3476,mount,wario,1
3477,jt,ruff23,1
3478,thunderstorm,warnings,1
3479,parts,ar,1


In [10]:
G = nx.from_pandas_edgelist(df, 'a', 'b', 'weight')

In [11]:
nx.write_graphml(G, '../data/processed/bigrams.graphml')